In [7]:
import numpy as np

lipsum = "Lorem ipsum dolor sit amet"
lipsum = lipsum.split(" ")

In [8]:
import pathlib
import sys
sys.path.insert(0, str(pathlib.Path().absolute().parent))

In [9]:
import pandas as pd

N = 7
nodes = [f"n{i}" for i in range(N)]

rnd = np.random.RandomState(seed=42)
df = pd.DataFrame(rnd.choice(lipsum, size=(N, 4), replace=True), index=nodes)
df.to_numpy()

array([['sit', 'amet', 'dolor', 'amet'],
       ['amet', 'ipsum', 'dolor', 'dolor'],
       ['dolor', 'amet', 'sit', 'dolor'],
       ['amet', 'ipsum', 'sit', 'ipsum'],
       ['sit', 'amet', 'Lorem', 'sit'],
       ['ipsum', 'amet', 'sit', 'Lorem'],
       ['Lorem', 'dolor', 'dolor', 'ipsum']], dtype=object)

In [11]:
from linkalike.utils.utils import pairwise_dissimilarities

dist_df = pairwise_dissimilarities(df, verbose=True)
dist_df.to_numpy().tolist()

100%|██████████| 7/7 [00:00<00:00, 9483.25it/s]


[[0, 3, 3, 4, 2, 3, 3],
 [3, 0, 3, 2, 4, 4, 3],
 [3, 3, 0, 3, 3, 2, 4],
 [4, 2, 3, 0, 4, 3, 3],
 [2, 4, 3, 4, 0, 3, 4],
 [3, 4, 2, 3, 3, 0, 4],
 [3, 3, 4, 3, 4, 4, 0]]